In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D1CE2568F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D1CE255630>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Manish, I'am a data science engineer")])

AIMessage(content="Hi Manish, it's nice to meet you!\n\nThat's a great field to be in. Data science engineering is becoming increasingly important as we generate more and more data. \n\nWhat kind of projects are you working on these days?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 24, 'total_tokens': 78, 'completion_time': 0.098181818, 'prompt_time': 0.000236999, 'queue_time': 0.013256391, 'total_time': 0.098418817}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-18c3368f-6620-43cb-9b6b-3eac7cd4ace3-0', usage_metadata={'input_tokens': 24, 'output_tokens': 54, 'total_tokens': 78})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Manish, I'am a data science engineer"),
        AIMessage(content="Hi Manish, it's nice to meet you!\n\nThat's a great field to be in. Data science engineering is becoming increasingly important as we generate more and more data. \n\nWhat kind of projects are you working on these days?"),
        HumanMessage(content="Hey, What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Manish and you are a data science engineer.  \n\nIs there anything else you'd like to talk about?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 98, 'total_tokens': 133, 'completion_time': 0.063636364, 'prompt_time': 0.00304355, 'queue_time': 0.01085123, 'total_time': 0.066679914}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-77a051f9-741c-4792-99aa-53c3bcf793df-0', usage_metadata={'input_tokens': 98, 'output_tokens': 35, 'total_tokens': 133})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable":{"session_id":"chat1"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Manish, I'am a data science engineer")],
    config=config
)

In [11]:
response.content

"Hi Manish, nice to meet you!\n\nIt's great to have a fellow data enthusiast here.  What kind of projects are you working on these days? Are you diving into machine learning, building data pipelines, or tackling some other exciting data challenges? \n\nI'm always curious to hear about what people are doing in the world of data science. 😊  \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config
)

AIMessage(content='Your name is Manish. 😊  I remember that from our first introduction! \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 193, 'total_tokens': 214, 'completion_time': 0.038181818, 'prompt_time': 0.005880951, 'queue_time': 0.017293946, 'total_time': 0.044062769}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-db5a8fc0-30d4-425f-8649-2fc39df229e7-0', usage_metadata={'input_tokens': 193, 'output_tokens': 21, 'total_tokens': 214})

In [14]:
## change the config --> session id
config1 = {"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)

response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it!\n"

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey, My Name id John")],
    config=config1
)

response.content

'Hi John, nice to meet you! \n\nIs there anything I can help you with today?\n'

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)

response.content

"You said your name is John!  \n\nRemember, I don't have a memory of past conversations unless you tell me again. 😄  Do you want to talk about something else, John?  \n\n"

### Prompt Templates

#### Prompt templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. let's now make that a bit more complicated. First lets add in a system message with some custom instructions(but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model


In [21]:
chain.invoke({"messages":[HumanMessage(content="HI. My name is Manish")]})

AIMessage(content="Hello Manish, it's nice to meet you!\n\nHow can I help you today?  😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 32, 'total_tokens': 59, 'completion_time': 0.049090909, 'prompt_time': 0.000342588, 'queue_time': 0.015023891000000001, 'total_time': 0.049433497}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-00716e34-c5b6-49df-afcc-09b1490ad0dc-0', usage_metadata={'input_tokens': 32, 'output_tokens': 27, 'total_tokens': 59})

In [23]:
config3 = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Bill")],
    config=config3
)

response.content

"Hello Bill, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n"

In [24]:
## Add more complexity.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [25]:
response = chain.invoke({"messages":[HumanMessage(content="Hi, My Name is Manish")], "language":"Hindi"})
response.content

'नमस्ते मनीष! 👋  मुझे बहुत खुशी है कि आपने मुझसे जुड़ा। 😊 \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपके सवालों के बेहतरीन जवाब दूं।  👍 \n\nक्या आप मुझसे कोई खास सवाल पूछना चाहते हैं? 🤔 \n'

#### Lets now wrap this more complicated chain in a message history class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [26]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [28]:
config4 = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, Iam Manish")], "language":"Hindi"},
    config=config4
)

response.content

'नमस्ते मनीष! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप मेरे से जो भी पूछना चाहें, मैं अपनी पूरी कोशिश करूँगा।  \n\nआपका कोई सवाल है?  \n'

In [29]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name")], "language":"Hindi"},
    config=config4
)

response.content

'आपका नाम मनीष है। 😊 \n'

## Managing the Conversation History
### One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
### 'trim_messages' helper to reduce how many messages we're sending to model. The trimmer allow us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [32]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi, I'am Bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | prompt | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What ice cream do I like")],
        "language":"English"
    }
)

response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor. \n\nWhat's your favorite flavor?  🍦😊\n"

In [35]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What maths problem did I ask")],
        "language":"English"
    }
)

response.content

'You asked "What\'s 2+2?".\n'

In [36]:
### Lets wrap this in message history.
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config5 = {"configurable":{"session_id":"chat5"}}

In [38]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Whats my name")],
    "language":"English"},
    config=config5
)

response.content

"As an AI, I have no memory of past conversations and don't know your name. Could you please tell me? 😊  \n\n"